# Similar Question finder
Find similar questions using word embeddings. Here we use Google's 300 dimensional word embeddings.

In [1]:
import gensim
import numpy as np
from gensim.models.keyedvectors import KeyedVectors
from sklearn.metrics.pairwise import cosine_similarity
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
import re
from tqdm import tqdm

C:\Users\SEEKER\Anaconda3\envs\tensorflow\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SEEKER\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
'''
    for preprocessing text
    text(string)
'''
def preprocess_text(text):
    STOPWORDS = set(stopwords.words('english'))
    # convert to lowercase
    text = text.lower()
    # replace whitespaces and punctuations
    text = re.sub('[/(){}\[\]\|@,;]', ' ', text)
    text = re.sub('[^0-9a-z #+_]', '', text)
    text = ' '.join(word for word in text.split() if word not in STOPWORDS)
    return text

In [3]:
'''
    For loading the data files
    filename(string): filename with full address
'''
def load_data(filename):
    data = []
    # open the file and load the training questions with positives and
    # negatives
    with open(filename, 'r', encoding='utf-8') as f:
        for line in f:
            data.append(line.strip().split('\t'))
    return data

In [10]:
'''
    For preprocessing the entire dataset
    data(list of list): list of questions and its candidate duplicates
'''
def preprocess_data(data):
    data_processed = []
    for i, line in tqdm(enumerate(data)):
        data_processed.append([])
        for j, l in enumerate(line):
            data_processed[i].append(preprocess_text(l))
            
    return data_processed

In [4]:
# load the data
train_data = data = load_data(filename='data/train.tsv')
data = load_data(filename='data/validation.tsv')

In [ ]:
# preprocess the data
data = preprocess_data(data)

9680it [45:18,  3.56it/s]

In [5]:
'''
    For ranking the duplicate question candidates
    question(string): original question
    candidates(list of list): list of duplicate candidates for each question
    embeddings(numpy array): embeddings 
'''
def rank_duplicate_candidates(question, candidates, embeddings, dim=300):
    # get embeddings for question 
    question_embed = question_embedding(question, word_embeddings, dim=dim)
    # get embeddings for candidates
    candidates_embed = np.array([question_embedding(candidate, word_embeddings) 
                          for candidate in candidates])
    # compute cosine similarity
    candidates_sim = cosine_similarity(question_embed.reshape(1,-1), candidates_embed)[0]
    # make a tuple pair(sim, candidate question index)
    candidates_ques_sim = [(sim, i) for i, sim in enumerate(candidates_sim)]
    # sort the list in descending order
    candidates_ques_sim = sorted(candidates_ques_sim, reverse=True)
    final_candidates_ranking = [(index, candidates[index]) for _,index in candidates_ques_sim]
    
    return final_candidates_ranking

In [6]:
# load google word embeddings
word_embeddings = KeyedVectors.load_word2vec_format(
                    'embeddings/GoogleNews-vectors-negative300.bin',
                    binary=True, limit=500000)

In [4]:
# sanity checking
print('neural' in word_embeddings)
print(word_embeddings.most_similar(positive=['woman', 'king'], negative=['man']))

True
[('queen', 0.7118192911148071), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321243286133), ('kings', 0.5236844420433044), ('queens', 0.518113374710083), ('sultan', 0.5098593235015869), ('monarchy', 0.5087411999702454), ('royal_palace', 0.5087165832519531)]


In [3]:
word_embeddings['apple'].shape

(300,)

In [7]:
''' For representing the question as vector of embeddings for 
    known words in question. We take the mean of all the embeddings for the 
    words of the question.
    
    question(string): input question
'''
def question_embedding(question, word_embeddings, dim=300):
    # tokenize the question 
    question = question.split()
    
    question_vector = [word_embeddings[word] for word in question if word in word_embeddings]
    
    # return if there is atleast one word known to the pretrained embedding
    if len(question_vector) == 0:
        return np.zeros(dim) 
    
    # take the mean along the axis = 0(along each column).
    # since each embedding is (300,1)
    # so all of them will have shape (no. of words, 300)
    question_vector = np.array(question_vector)
    question_vector = np.mean(question_vector, axis=0)
    
    return question_vector 

## <u>Evaluation metrics
We will use two evaluation metrics for checking the quality of the predicting the duplicate.
    
***1. Hits@k***
<br>It sees how many times the correct duplicate candiate lie within the first k entries.
    

$$ \text{Hits@K} = \frac{1}{N}\sum_{i=1}^N \, [dup_i \in topK(q_i)]$$

$q_i:$ i-th query <br>$dup_i:$ Correct duplicate for this question<br> $topK(q_i):$ Top K ranked duplicate candidates by the model<br>
$dup_i \in topK(q_i)$ equals 1 if the condition is true and 0 otherwise.

In [8]:
'''
    ranks: list of ranks for each duplicate
    k: (int) stating the window size to scan
'''
def hits_k(k, ranks):
    hit_score = np.array(ranks) <= k
    hit_score = hit_score.mean()
    return hit_score


***2. DCG@k***
If the position of the correct duplicate is higher then the score will be higher.

$$ \text{DCG@K} = \frac{1}{N} \sum_{i=1}^N\frac{1}{\log_2(1+rank_{dup_i})}\cdot[rank_{dup_i} \le K] $$

$rank_{dup_i}:$ Position of the correct duplicate in the sorted list of candidates for the current query $q_i$ question. <br>

In [9]:
'''
    ranks: list of ranks for each duplicate
    k: (int) stating the window size to scan
'''
def dcg_k(k, ranks):
    dcg_score = np.array(ranks) <= k
    dcg_score = dcg_score/np.log2(1 + np.array(ranks))
    dcg_score = dcg_score.mean()
    
    return dcg_score

In [69]:
# for storing the ranks of the duplicates
duplicate_rank = []

for line in data_processed:
    # get the question and duplicate candidates
    question, *candidates = line
    # get the rankings for the candidates
    candidate_rankings = rank_duplicate_candidates(question, candidates, word_embeddings)
    # store the index of the correct duplicate in rankings
    for rank,(index,_) in enumerate(candidate_rankings):
        if index == 0:
            duplicate_rank.append(rank+1)
            break

In [73]:
# check scores for different k values
print("    K|    DCG Score   |    Hits Score")
for k in [1, 5, 10, 50, 100, 200, 500, 1000]:
    print("%5d|    %.4f      |    %.4f" % 
          (k, dcg_k(k, duplicate_rank), hits_k(k, duplicate_rank)))

    K|    DCG Score   |    Hits Score
    1|    0.2944      |    0.2944
    5|    0.3706      |    0.4374
   10|    0.3873      |    0.4890
   50|    0.4122      |    0.6009
  100|    0.4212      |    0.6562
  200|    0.4297      |    0.7172
  500|    0.4433      |    0.8304
 1000|    0.4610      |    1.0000
